In [163]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [164]:
a = 2.0
b = -3.0
c = 8.0
d = a*b + c
print(d)

2.0


Understanding micrograd and basic NN. The back propogation essentially just locates derivatives of the function applying the chain rule.

In [165]:
class Value_T:
    def __init__(self,data,_children=()):
        self.data = data
        self._prev = set(_children)
    def __repr__(self):
        return f"Value(data={self.data})"
    
a = Value_T(2.0)
b = Value_T(-3.0)
a + b

TypeError: unsupported operand type(s) for +: 'Value_T' and 'Value_T'

Results in error as the wrapping of data as Values is uknown to Python... must define operations within the class

In [170]:
class Value:
    def __init__(self,data,_children=(),_op='',label=''): 
        #children tracks the values used to keep track of necessary builders
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self,other): #+/- nopes 'route' derivatives
        out = Value(self.data + other.data, (self,other),'+')

        def _backward():
            self.grad += other.data*out.grad
            other.grad += self.data*out.grad
        out._backward = _backward

        return out
#by using __ operators, it changes the default operations
    def __mul__(self,other): #*/ nodes 
        out = Value(self.data*other.data, (self,other),'*')
        
        def _backward():
            self.grad += other.data*out.grad
            other.grad += self.data*out.grad
        out._backward = _backward

        return out
    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,),'tanh')

        def _backward():
            self.grad += (1-t**2)*out.grad #derivative of tan function
        out._backward = _backward

        return out
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)
e = a*b; e.label='e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d*f; L.label = 'L'
L
#(a.__mul__(b)).__add__(c)

Value(data=-8.0)

In [171]:
d._prev

{Value(data=-6.0), Value(data=10.0)}

In [172]:
d._op

'+'

When applied to Neural Networks it is crucial to find the way the Gradient changes each step akong the function... Hence the addition of grad!

Below is an application to an actual Neuron with a tanh 'squashing function'

In [173]:
#inputs x1,x2
x1 = Value(2.0,label='x1')
x2 = Value(0.0,label='x2')
#weights w1,w2
w1 = Value(-3.0,label='w1')
w2 = Value(1.0,label='w2')
#bias of the neuron (f)
b = Value(6.8813735870195432,label='b')
# x1w1 + x2w2 + b ; is the total function by the end
x1w1 = x1*w1; x1w1.label = 'x1w1'
x2w2 = x2*w2; x2w2.label = 'x2w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'

n

Value(data=0.8813735870195432)

In [174]:
o = n.tanh(); o.label = 'o'
o

Value(data=0.7071067811865476)

In [175]:
o.grad = 1.0
o._backward()
n._backward()
n.grad

0.4999999999999999

Because we defined the backward function within the class above I can now run o.backward() directly without needing to _backward() each step...

In [176]:
o.backward()

which does this...

In [177]:
def build_topo(v):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(v)
        return topo
build_topo(o)

[Value(data=6.881373587019543),
 Value(data=0.0),
 Value(data=1.0),
 Value(data=0.0),
 Value(data=-3.0),
 Value(data=2.0),
 Value(data=-6.0),
 Value(data=-6.0),
 Value(data=0.8813735870195432),
 Value(data=0.7071067811865476)]